# Custom layers

We recommend using tf.keras as a high-level API for building neural networks. That said, most TensorFlow APIs are usable with eager execution.



In [1]:
import tensorflow as tf
tfe = tf.contrib.eager

tf.enable_eager_execution()

## Layers: common sets of useful operations

Most of the time when writing code for machine learning models you want to operate at a higher level of abstraction than individual operations and manipulation of individual variables.

Many machine learning models are expressible as the composition and stacking of relatively simple layers, and TensorFlow provides both a set of many common layers as a well as easy ways for you to write your own application-specific layers either from scratch or as the composition of existing layers.

TensorFlow includes the full Keras API in the tf.keras package, and the Keras layers are very useful when building your own models.

In [2]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to 
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

The full list of pre-existing layers can be seen in the [documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers). It includes Dense (a fully-connected layer), Conv2D, LSTM, BatchNormalization, Dropout, and many others.

In [3]:
# To use a layer, simply call it.
layer(tf.zeros([10, 5]))

<tf.Tensor: id=31, shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [4]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer by calling layer.variables. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.18417025, -0.56486297,  0.27888966,  0.35119605, -0.13550666,
         -0.44338417,  0.33006084,  0.06465781,  0.46478242, -0.24840882],
        [ 0.1292625 , -0.5518681 , -0.05588156,  0.46745163, -0.39220953,
          0.5647306 ,  0.4595632 , -0.15694058, -0.31543523, -0.19423813],
        [ 0.39873916,  0.28413367,  0.6045758 , -0.2998222 , -0.42489004,
          0.3668676 ,  0.14631432,  0.0965696 ,  0.01653373, -0.38958108],
        [-0.6121994 ,  0.54210716, -0.626118  ,  0.21551025,  0.11278081,
         -0.23054561,  0.39739722,  0.55595225,  0.16450524,  0.54947716],
        [ 0.04030669,  0.50555485,  0.56921   , -0.5033162 ,  0.3607558 ,
         -0.03648508,  0.48027354, -0.56356615,  0.04034936, -0.02477765]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

## Implementing custom layers

The best way to implement your own layer is extending the tf.keras.Layer class and implementing: * _\_init_\_ , where you can do all input-independent initialization * build, where you know the shapes of the input tensors and can do the rest of the initialization * call, where you do the forward computation

Note that you don't have to wait until build is called to create your variables, you can also create them in _\_init_\_. However, the advantage of creating them in build is that it enables late variable creation based on the shape of the inputs the layer will operate on. On the other hand, creating variables in _\_init_\_ would mean that shapes required to create the variables will need to be explicitly specified.

In [5]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self,num_outputs):
        super(MyDenseLayer,self).__init__()
        self.num_outputs = num_outputs
        
    def build(self,input_shape):
        self.kernel = self.add_variable("kernel",
                                       shape=[input_shape[-1].value,
                                             self.num_outputs])
    
    def call(self,input):
        return tf.matmul(input,self.kernel)

layer = MyDenseLayer(10)
print(layer(tf.zeros([10,5])))
print(layer.variables)

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(10, 10), dtype=float32)
[<tf.Variable 'my_dense_layer/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[ 0.32040465, -0.06526864, -0.41416126,  0.13542253, -0.28242174,
        -0.5784014 ,  0.3198794 , -0.41260964,  0.5026553 ,  0.08591753],
       [ 0.62458795, -0.31851944,  0.41520518, -0.24026531,  0.02483344,
         0.09086478,  0.60897917,  0.5206612 , -0.513631  ,  0.33434027],
       [-0.41990706,  0.53487974,  0.61063915, -0.23088685, -0.22634432,
        -0.01263779,  0.44161004,  0.39503866, -0.23409382,  0.41845566],
       [ 0.6287927 ,  0.04924351, -0.24617189, -0.3197214 , -0.49019244,
         0.02609271,  0.05449581,  0

Note that you don't have to wait until build is called to create your variables, you can also create them in _\_init_\_.

Overall code is easier to read and maintain if it uses standard layers whenever possible, as other readers will be familiar with the behavior of standard layers. If you want to use a layer which is not present in tf.keras.layers or tf.contrib.layers, consider filing a github issue or, even better, sending us a pull request!

## Models: composing layers

Many interesting layer-like things in machine learning models are implemented by composing existing layers. For example, each residual(剩余的) block in a resnet is a composition of convolutions, batch normalizations, and a shortcut.

The main class used when creating a layer-like thing which contains other layers is tf.keras.Model. Implementing one is done by inheriting from tf.keras.Model.

In [10]:
class ResnetIdentityBlock(tf.keras.Model):
    def __init__(self,kernel_size,filters):
        super(ResnetIdentityBlock,self).__init__(name='')
        filters1, filters2, filters3 = filters
        
        self.conv2a = tf.keras.layers.Conv2D(filters1,(1,1))
        self.bn2a = tf.keras.layers.BatchNormalization()
        
        self.conv2b = tf.keras.layers.Conv2D(filters2,kernel_size,padding='same')
        self.bn2b = tf.keras.layers.BatchNormalization()
        
        self.conv2c = tf.keras.layers.Conv2D(filters3,(1,1))
        self.bn2c = tf.keras.layers.BatchNormalization()
    
    def call(self,input_tensor,training=False):
        x = self.conv2a(input_tensor)
        x = self.bn2a(x,training=training)
        x = tf.nn.relu(x)
        
        x = self.conv2b(x)
        x = self.bn2b(x,training=training)
        x = tf.nn.relu(x)
        
        x = self.conv2c(x)
        x = self.bn2c(x, training=training)
        
        x += input_tensor
        return tf.nn.relu(x)

block = ResnetIdentityBlock(1,[1,2,3])
print(block(tf.zeros([1,2,3,3])))
print([x.name for x in block.variables])

tf.Tensor(
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], shape=(1, 2, 3, 3), dtype=float32)
['resnet_identity_block_4/conv2d_8/kernel:0', 'resnet_identity_block_4/conv2d_8/bias:0', 'resnet_identity_block_4/batch_normalization_8/gamma:0', 'resnet_identity_block_4/batch_normalization_8/beta:0', 'resnet_identity_block_4/conv2d_9/kernel:0', 'resnet_identity_block_4/conv2d_9/bias:0', 'resnet_identity_block_4/batch_normalization_9/gamma:0', 'resnet_identity_block_4/batch_normalization_9/beta:0', 'resnet_identity_block_4/conv2d_10/kernel:0', 'resnet_identity_block_4/conv2d_10/bias:0', 'resnet_identity_block_4/batch_normalization_10/gamma:0', 'resnet_identity_block_4/batch_normalization_10/beta:0', 'resnet_identity_block_4/batch_normalization_8/moving_mean:0', 'resnet_identity_block_4/batch_normalization_8/moving_variance:0', 'resnet_identity_block_4/batch_normalization_9/moving_mean:0', 'resnet_identity_block_4/batch_normalization_9/moving_variance:

Much of the time, however, models which compose many layers simply call one layer after the other. This can be done in very little code using tf.keras.Sequential

In [11]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1,(1,1)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2,1,
                                                   padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3,(1,1)),
                             tf.keras.layers.BatchNormalization()])

my_seq(tf.zeros([1,2,3,3]))

<tf.Tensor: id=544, shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>